<a href="https://colab.research.google.com/github/PSN-01/NFL-WRS-RBS-TES-Individual-Stats/blob/main/WRS_RBS_TES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DOWNLOAD THEM FIRST!
!pip install nfl_data_py
!pip install pandas



*   **Names to input are in this format (DONT FORGET CAPITAL LETTERS or dots): C.Kupp (NO JR's) ex. M.Harrison (ARI), O.beckham (NYG) or B.Thomas (JAX)**
*   **Teams are input in team_abbr**
*   **nfl_data_py only works for 1999-2024**

In [ ]:

import nfl_data_py as nfl

# Import team descriptions
teams = nfl.import_team_desc()

# Team names and team abbreviation
print(teams[['team_abbr', 'team_name']])


# ***`Wide receivers, Tigh ends & Running backs (since 2016 with route stats)`***

In [21]:
import nfl_data_py as nfl

def wr_stats(player_name, team_name, year):

    pbp = nfl.import_pbp_data([year])

    # Receiving Filters
    receiving_data = pbp[(pbp['season_type'] == 'REG') &  # Removes Playoff games
                         (pbp['receiver_player_name'] == player_name) &
                         (pbp['posteam'] == team_name)]  # Filter by team

    receptions = receiving_data[receiving_data['complete_pass'] == 1].shape[0]
    total_receiving_yards = receiving_data[receiving_data['complete_pass'] == 1]['yards_gained'].sum()
    total_receiving_touchdowns = receiving_data[receiving_data['pass_touchdown'] == 1].shape[0]
    average_receiving_yards = total_receiving_yards / receptions if receptions > 0 else 0

    # Top 5 Routes
    if 'route' in receiving_data.columns:
        route_counts = receiving_data['route'].value_counts()
        top_routes = route_counts.head(5)
        total_routes = receiving_data['route'].notnull().sum()  # Count all routes
        top_routes_percentage = (top_routes / total_routes) * 100 if total_routes > 0 else 0


    # Rushing filters
    rushing_data = pbp[(pbp['season_type'] == 'REG') &  # Remove Playoff games
                       (pbp['rusher_player_name'] == player_name) &
                       (pbp['posteam'] == team_name)]  # Filter by team

    rush_attempts = rushing_data.shape[0]
    total_rushing_yards = rushing_data['yards_gained'].sum()
    total_rushing_touchdowns = rushing_data['rush_touchdown'].sum()

    # 2pt conversions
    two_point_conversion_data = pbp[(pbp['season_type'] == 'REG') &
                                    (pbp['two_point_attempt'] == 1) &
                                    (pbp['two_point_conv_result'] == 'success') &
                                    ((pbp['receiver_player_name'] == player_name) |
                                     (pbp['rusher_player_name'] == player_name)) &
                                    (pbp['posteam'] == team_name)]  # Filter by team

    two_point_conversions = two_point_conversion_data.shape[0]

    # Starting games
    games_played = pbp[(pbp['season_type'] == 'REG') &
                       ((pbp['receiver_player_name'] == player_name) |
                        (pbp['rusher_player_name'] == player_name)) &
                       (pbp['posteam'] == team_name)]['game_id'].nunique()

    print(f"\nReceiving Stats for {player_name} in {year}:\n")
    print(f"Total Receptions: {receptions}")
    print(f"Total Receiving Yards: {total_receiving_yards}")
    print(f"Total Receiving Touchdowns: {total_receiving_touchdowns}")
    print(f"Average Receiving Yards: {average_receiving_yards:.2f}")


    print(f"\nRushing Stats for {player_name}:\n")
    print(f"Total Rushing Attempts: {rush_attempts}")
    print(f"Total Rushing Yards: {total_rushing_yards}")
    print(f"Total Rushing Touchdowns: {total_rushing_touchdowns}")

    # Routes only available from 2016 onwars
    if 'route' in receiving_data.columns:
        print(f"\nTop 5 Most Run Routes for {player_name}:\n")
        for route, count in top_routes.items():
            percentage = top_routes_percentage[route]
            print(f"{route}: {count} times, ({percentage:.2f}%)")

    print(f"\nOther Stats for {player_name}:\n")
    print(f"Total Two-Point Conversions: {two_point_conversions}")

    # Calculate Fantasy PPR Points NOT COUNTING FUMBLES
    ppr_points = receptions + (total_receiving_touchdowns * 6) + (total_receiving_yards / 10) + (total_rushing_touchdowns * 6) + (total_rushing_yards / 10) + (two_point_conversions * 2)
    ppr_points_per_game = ppr_points / games_played if games_played > 0 else 0

    print(f"Fantasy PPR Points: {ppr_points:.2f} (WITHOUT FUMBLES & PASSING STATS)")
    print(f"Fantasy PPR Points Per Game: {ppr_points_per_game:.2f} (Average)")

player_name = input("Enter the player's name: ")
team_name = input("Enter the player's team: ")
year = int(input("Enter the player's year: "))
wr_stats(player_name, team_name, year)

Enter the player's name: B.Thomas
Enter the player's team: JAX
Enter the player's year: 2024
2024 done.
Downcasting floats.

Receiving Stats for B.Thomas in 2024:

Total Receptions: 22
Total Receiving Yards: 397.0
Total Receiving Touchdowns: 3
Average Receiving Yards: 18.05

Rushing Stats for B.Thomas:

Total Rushing Attempts: 1
Total Rushing Yards: 13.0
Total Rushing Touchdowns: 0.0

Other Stats for B.Thomas:

Total Two-Point Conversions: 0
Fantasy PPR Points: 81.00 (WITHOUT FUMBLES & PASSING STATS)
Fantasy PPR Points Per Game: 16.20 (Average)
